# Ridge and Lasso Regression - Lab

## Introduction

In this lab, you'll practice your knowledge of Ridge and Lasso regression!

## Objectives

In this lab you will: 

- Use Lasso and Ridge regression with scikit-learn 
- Compare and contrast Lasso, Ridge and non-regularized regression 

## Housing Prices Data

Let's look at yet another house pricing dataset: 

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('Housing_Prices/train.csv')

Look at `.info()` of the data: 

In [12]:
# Your code here
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

- First, split the data into `X` (predictor) and `y` (target) variables 
- Split the data into 75-25 training-test sets. Set the `random_state` to 10 
- Remove all columns of `object` type from `X_train` and `X_test` and assign them to `X_train_cont` and `X_test_cont`, respectively 

In [13]:
# Create X and y
y = df[['SalePrice']]
X = df.drop(columns='SalePrice')

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.25,random_state=10)

# Remove "object"-type features from X
cont_features = X.select_dtypes(exclude=['object'])

# Remove "object"-type features from X_train and X_test
X_train_cont = X_train.select_dtypes(exclude=['object'])
X_test_cont = X_test.select_dtypes(exclude=['object'])

## Let's use this data to build a first naive linear regression model

- Fill the missing values in data using median of the columns (use [`SimpleImputer`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html)) 
- Fit a linear regression model to this data 
- Compute the R-squared and the MSE for both the training and test sets 


In [14]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer

# Impute missing values with median using SimpleImputer
impute = SimpleImputer(strategy='median')
X_train_imputed = impute.fit_transform(X_train_cont)
X_test_imputed = impute.fit_transform(X_test_cont)

# Fit the model and print R2 and MSE for training and test sets
linreg = LinearRegression()
linreg.fit(X_train_imputed, y_train)

# Print R2 and MSE for training and test sets
print('R squared for training is: ',linreg.score(X_train_imputed, y_train))
print('MSE for training is: ', mean_squared_error(y_train, linreg.predict(X_train_imputed)))
print('R squared for testing is: ',linreg.score(X_test_imputed, y_test))
print('MSE for testing is: ', mean_squared_error(y_test, linreg.predict(X_test_imputed)))

R squared for training is:  0.8069714678400265
MSE for training is:  1212415985.7084064
R squared for testing is:  0.8202362411915377
MSE for testing is:  1146926057.153154


## Normalize your data

- Normalize your data using a `StandardScalar`  
- Fit a linear regression model to this data 
- Compute the R-squared and the MSE for both the training and test sets 


In [15]:
from sklearn.preprocessing import StandardScaler

# Scale the train and test data
ss = StandardScaler()
X_train_imputed_scaled = ss.fit_transform(X_train_imputed)
X_test_imputed_scaled = ss.transform(X_test_imputed)

# Fit the model
linreg_norm = LinearRegression()
linreg_norm.fit(X_train_imputed_scaled,y_train)


# Print R2 and MSE for training and test sets
print('R squared for training is: ',linreg_norm.score(X_train_imputed_scaled, y_train))
print('R squared for testing is: ',linreg_norm.score(X_test_imputed_scaled, y_test))
print('MSE for training is: ', mean_squared_error(y_train, linreg_norm.predict(X_train_imputed_scaled)))
print('MSE for testing is: ', mean_squared_error(y_test, linreg_norm.predict(X_test_imputed_scaled)))

R squared for training is:  0.8070159754195584
R squared for testing is:  0.8201539233754344
MSE for training is:  1212136432.7308965
MSE for testing is:  1147451260.0576918


## Include categorical variables

The above models didn't include categorical variables so far, let's include them! 


- Include all columns of `object` type from `X_train` and `X_test` and assign them to `X_train_cat` and `X_test_cat`, respectively 
- Fill missing values in all these columns with the string `'missing'` 

In [16]:
# Create X_cat which contains only the categorical variables
features_cat = [var for var in df if df[var].dtype == 'object']
X_train_cat = X_train[features_cat]
X_test_cat = X_test[features_cat]

# Fill missing values with the string 'missing'
X_train_cat.fillna(value='missing',inplace=True)
X_test_cat.fillna(value='missing',inplace=True)


- One-hot encode all these categorical columns using `OneHotEncoder` 
- Transform the training and test DataFrames (`X_train_cat`) and (`X_test_cat`) 
- Run the given code to convert these transformed features into DataFrames 

In [17]:
from sklearn.preprocessing import OneHotEncoder

# OneHotEncode categorical variables
ohe = OneHotEncoder(handle_unknown='ignore')

# Transform training and test sets
X_train_ohe = ohe.fit_transform(X_train_cat)
X_test_ohe = ohe.transform(X_test_cat)

# Convert these columns into a DataFrame
columns = ohe.get_feature_names(input_features=X_train_cat.columns)
cat_train_df = pd.DataFrame(X_train_ohe.todense(), columns=columns)
cat_test_df = pd.DataFrame(X_test_ohe.todense(), columns=columns)

- Combine `X_train_imputed_scaled` and `cat_train_df` into a single DataFrame  
- Similarly, combine `X_test_imputed_scaled` and `cat_test_df` into a single DataFrame 

In [18]:
# Your code here
X_train_all = pd.concat([pd.DataFrame(X_train_imputed_scaled),cat_train_df],axis=1)
X_test_all = pd.concat([pd.DataFrame(X_test_imputed_scaled),cat_test_df],axis=1)

Now build a linear regression model using all the features (`X_train_all`). Also, print the R-squared and the MSE for both the training and test sets. 

In [19]:
# Your code here
linmod = LinearRegression()
linmod.fit(X_train_all,y_train)

print('R squared for training is: ',linmod.score(X_train_all, y_train))
print('R squared for testing is: ',linmod.score(X_test_all, y_test))
print('MSE for training is: ', mean_squared_error(y_train, linmod.predict(X_train_all)))
print('MSE for testing is: ', mean_squared_error(y_test, linmod.predict(X_test_all)))


R squared for training is:  0.9359959790142014
R squared for testing is:  -3.38029315689305e+19
MSE for training is:  402010507.5890411
MSE for testing is:  2.1566896064895968e+29


Notice the severe overfitting above; our training R-squared is very high, but the test R-squared is negative! Similarly, the scale of the test MSE is orders of magnitude higher than that of the training MSE.

## Ridge and Lasso regression

Use all the data (normalized features and dummy categorical variables, `X_train_all`) to build two models - one each for Lasso and Ridge regression. Each time, look at R-squared and MSE. 

In [20]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

## Lasso

#### With default parameter (alpha = 1)

In [21]:
# Your code here
lasso1 = Lasso()
lasso1.fit(X_train_all,y_train)

print('R squared for training is: ',lasso1.score(X_train_all, y_train))
print('R squared for testing is: ',lasso1.score(X_test_all, y_test))
print('MSE for training is: ', mean_squared_error(y_train, lasso1.predict(X_train_all)))
print('MSE for testing is: ', mean_squared_error(y_test, lasso1.predict(X_test_all)))

R squared for training is:  0.9359681086176651
R squared for testing is:  0.8886774108036876
MSE for training is:  402185562.09476924
MSE for testing is:  710258725.926221


#### With a higher regularization parameter (alpha = 10)

In [22]:
# Your code here
lasso2 = Lasso(alpha=10)
lasso2.fit(X_train_all,y_train)

print('R squared for training is: ',lasso2.score(X_train_all, y_train))
print('R squared for testing is: ',lasso2.score(X_test_all, y_test))
print('MSE for training is: ', mean_squared_error(y_train, lasso2.predict(X_train_all)))
print('MSE for testing is: ', mean_squared_error(y_test, lasso2.predict(X_test_all)))

R squared for training is:  0.9343826511712741
R squared for testing is:  0.8966769715994979
MSE for training is:  412143851.32359606
MSE for testing is:  659220047.2553351


## Ridge

#### With default parameter (alpha = 1)

In [23]:
# Your code here
ridge1 = Ridge()
ridge1.fit(X_train_all,y_train)

print('R squared for training is: ',ridge1.score(X_train_all, y_train))
print('R squared for testing is: ',ridge1.score(X_test_all, y_test))
print('MSE for training is: ', mean_squared_error(y_train, ridge1.predict(X_train_all)))
print('MSE for testing is: ', mean_squared_error(y_test, ridge1.predict(X_test_all)))

R squared for training is:  0.9231940244796031
R squared for testing is:  0.8842262846359505
MSE for training is:  482419834.3987994
MSE for testing is:  738657734.8215194


#### With default parameter (alpha = 10)

In [24]:
# Your code here
ridge2 = Ridge(alpha=10)
ridge2.fit(X_train_all,y_train)

print('R squared for training is: ',ridge2.score(X_train_all, y_train))
print('R squared for testing is: ',ridge2.score(X_test_all, y_test))
print('MSE for training is: ', mean_squared_error(y_train, ridge2.predict(X_train_all)))
print('MSE for testing is: ', mean_squared_error(y_test, ridge2.predict(X_test_all)))

R squared for training is:  0.8990002650425939
R squared for testing is:  0.8834211676790178
MSE for training is:  634381310.5991349
MSE for testing is:  743794530.0414388


## Compare the metrics    

Write your conclusions here: 
_________________________________


Looks like the Lasso model with an alpha of 10 produces the best results

## Compare number of parameter estimates that are (very close to) 0 for Ridge and Lasso

Use 10**(-10) as an estimate that is very close to 0. 

In [25]:
# Number of Ridge params almost zero


In [26]:
# Number of Lasso params almost zero

In [27]:
print(len(lasso2.coef_))
print(sum(abs(lasso2.coef_) < 10**(-10))/ len(lasso2.coef_))

296
0.26013513513513514


Lasso was very effective to essentially perform variable selection and remove about 25% of the variables from your model!

## Put it all together

To bring all of our work together lets take a moment to put all of our preprocessing steps for categorical and continuous variables into one function. This function should take in our features as a dataframe `X` and target as a Series `y` and return a training and test DataFrames with all of our preprocessed features along with training and test targets. 

In [30]:
def preprocess(X, y):
    '''Takes in features and target and implements all preprocessing steps for categorical and continuous features returning 
    train and test DataFrames with targets'''
    
    # Train-test split (75-25), set seed to 10
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.25,random_state=10)
    
    # Remove "object"-type features and SalesPrice from X
    continous = [var for var in X if X[var].dtype != 'object']
    X_con = X[continous]
    X_train_con = X_train[continous]
    X_test_con = X_test[continous]

    # Impute missing values with median using SimpleImputer
    impute = SimpleImputer(strategy='median')
    X_train_imputed = impute.fit_transform(X_train_cont)
    X_test_imputed = impute.fit_transform(X_test_cont)
    # Scale the train and test data
    ss = StandardScaler()
    X_train_imputed_scaled = ss.fit_transform(X_train_imputed)
    X_test_imputed_scaled = ss.transform(X_test_imputed)

    # Create X_cat which contains only the categorical variables
    categorical = [var for var in X if X[var].dtype == 'object']
    X_cat = X[categorical]
    X_train_cat = X_train[categorical]
    X_test_cat = X_test[categorical]


    # Fill nans with a value indicating that that it is missing
    
    X_train_cat.fillna(value='missing',inplace=True)
    X_test_cat.fillna(value='missing',inplace=True)

    # OneHotEncode Categorical variables
    ohe = OneHotEncoder(handle_unknown='ignore')

    # Transform training and test sets
    X_train_ohe = ohe.fit_transform(X_train_cat)
    X_test_ohe = ohe.transform(X_test_cat)
    
    # Combine categorical and continuous features into the final dataframe
    columns = ohe.get_feature_names(input_features=X_train_cat.columns)
    cat_train_df = pd.DataFrame(X_train_ohe.todense(), columns=columns)
    cat_test_df = pd.DataFrame(X_test_ohe.todense(), columns=columns)
    
    X_train_all = pd.concat([pd.DataFrame(X_train_imputed_scaled),cat_train_df],axis=1)
    X_test_all = pd.concat([pd.DataFrame(X_test_imputed_scaled),cat_test_df],axis=1)
    
    return X_train_all, X_test_all, y_train, y_test

### Graph the training and test error to find optimal alpha values

Earlier we tested two values of alpha to see how it effected our MSE and the value of our coefficients. We could continue to guess values of alpha for our Ridge or Lasso regression one at a time to see which values minimize our loss, or we can test a range of values and pick the alpha which minimizes our MSE. Here is an example of how we would do this:  

In [ ]:
X_train_all, X_test_all, y_train, y_test = preprocess(X, y)

train_mse = []
test_mse = []
alphas = []

for alpha in np.linspace(0, 200, num=50):
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train_all, y_train)
    
    train_preds = lasso.predict(X_train_all)
    train_mse.append(mean_squared_error(y_train, train_preds))
    
    test_preds = lasso.predict(X_test_all)
    test_mse.append(mean_squared_error(y_test, test_preds))
    
    alphas.append(alpha)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots()
ax.plot(alphas, train_mse, label='Train')
ax.plot(alphas, test_mse, label='Test')
ax.set_xlabel('Alpha')
ax.set_ylabel('MSE')

# np.argmin() returns the index of the minimum value in a list
optimal_alpha = alphas[np.argmin(test_mse)]

# Add a vertical line where the test MSE is minimized
ax.axvline(optimal_alpha, color='black', linestyle='--')
ax.legend();

print(f'Optimal Alpha Value: {int(optimal_alpha)}')

Take a look at this graph of our training and test MSE against alpha. Try to explain to yourself why the shapes of the training and test curves are this way. Make sure to think about what alpha represents and how it relates to overfitting vs underfitting.

## Summary

Well done! You now know how to build Lasso and Ridge regression models, use them for feature selection and find an optimal value for $\text{alpha}$. 